# JSNAPy 

JSNAPy is a Juniper tool used to check whether or not the status of a network device has changed.  

It can be used to regularly check the health of deployed devices against known good states or it could be used to check the status of a device before and after a maintainance window.  That way you can easily check to make sure there have been no unforseen side effects caused by your maintainance.

It can be used, either as a standalone CLI based tool or within a Python script.  You use it as a standalone tool as follows:

In [1]:
# Configuration
# Here you specify the devices you want to inspect and the tests you want to run
!cat ./Section-3_JSNAPy/config.yml

hosts:
  - device: '172.12.1.2'
    username: netconf
    passwd: netconf123
tests:
  - ./Section-3_JSNAPy/check_sw_ver.yml
  - ./Section-3_JSNAPy/check_bgp_nei.yml


In [2]:
# Tests
# The first test compares software versions before and after.  It uses a CLI command to get the device data
!cat ./Section-3_JSNAPy/check_sw_ver.yml
# The second test checks that all BGP sessions are Established. It uses an RPC call to get the device data
!cat ./Section-3_JSNAPy/check_bgp_nei.yml
# There are many different operators availale

---
test_sw_version:
  - command: show version
  - item: 
      xpath: '//software-information/package-information'
      tests:
        - all-same: comment
          err: "Test failed. <{{pre['comment']}}> not equal to <{{post['comment']}}>"
          info: "Success, versions are equal"
---
test_bgp_nei:
  - rpc: get-bgp-neighbor-information
  - iterate:
      xpath: '//bgp-information/bgp-peer'
      tests:
        - is-equal: peer-state, Established     
          err: "BGP neighbour issue detected. {{pre['peer-address']}}. State was: {{pre['peer-state']}}. Now state is {{post['peer-state']}}"


Once the tests have been configured you take a first snapshot:

In [3]:
!jsnapy -v --snap pre_maintainance -f ./Section-3_JSNAPy/config.yml

Traceback (most recent call last):
  File "/opt/anaconda3/bin/jsnapy", line 8, in <module>
    sys.exit(main())
  File "/opt/anaconda3/lib/python3.9/site-packages/jnpr/jsnapy/jsnapy.py", line 1114, in main
    js = SnapAdmin()
  File "/opt/anaconda3/lib/python3.9/site-packages/jnpr/jsnapy/jsnapy.py", line 44, in __init__
    setup_logging.setup_logging()
  File "/opt/anaconda3/lib/python3.9/site-packages/jnpr/jsnapy/setup_logging.py", line 20, in setup_logging
    config_location = get_config_location("logging.yml")
  File "/opt/anaconda3/lib/python3.9/site-packages/jnpr/jsnapy/__init__.py", line 65, in get_config_location
    raise FileNotFoundError("Could not locate %s" % file)
FileNotFoundError: Could not locate logging.yml


Then you perform the maintainance as normal, once you are finished you take a second snapshot.

In [ ]:
!jsnapy -v --snap post_maintainance -f ./Section-3_JSNAPy/config.yml

You can then compare the snapshots, if any tests fail you will be notified.

In [ ]:
!jsnapy --check  pre_maintainance post_maintainance -f ./Section-3_JSNAPy/config.yml 

## JSNAPy in a Python Script

If you wanted to combine pushing configuration via automation with JSNAPy pre and post checks, you can do this with the JSNAPy libraries:

In [1]:
from pprint import pprint
from jnpr.jsnapy import SnapAdmin

with open('./Section-3_JSNAPy/config.yml') as fh:
    jsnapy_config = fh.read()

js = SnapAdmin()
js.snap(jsnapy_config, 'py_pre')
# You could insert your maintainance code here
js.snap(jsnapy_config, 'py_post')

snapchk = js.check(jsnapy_config, pre_file='py_pre', post_file='py_post')

for val in snapchk:
    pprint(dict(val.test_details))
    # You can easily take automated actions based on what the results of the tests were

ModuleNotFoundError: No module named 'jnpr.jsnapy'